<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Peer Review Assignment - Data Engineer - ETL


Estimated time needed: **20** minutes


## Objectives

In this final part you will:

*   Run the ETL process
*   Extract bank and market cap data from the JSON file `bank_market_cap.json`
*   Transform the market cap currency using the exchange rate data
*   Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment or in SN Labs. Others may need to be installed by you. The cells below will install these libraries when executed.


In [37]:
#!mamba install pandas==1.3.3 -y
#!mamba install requests==2.26.0 -y

## Imports

Import any additional libraries you may need here.


In [38]:
import glob
import pandas as pd
from datetime import datetime

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section


In [39]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2023-01-24 09:13:31--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json.1’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2023-01-24 09:13:32 (83.5 MB/s) - ‘bank_market_cap_1.json.1’ saved [2815/2815]

--2023-01-24 09:13:33--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%2

In [40]:
pd.read_json('bank_market_cap_1.json')

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399
...,...,...
65,Ping An Bank,37.993
66,Standard Chartered,37.319
67,United Overseas Bank,35.128
68,QNB Group,33.560


## Extract


### JSON Extract Function

This function will extract JSON files.


In [41]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [42]:
columns=['Name','Market Cap (US$ Billion)']

In [43]:
def extract():
    extracted_data = pd.DataFrame(extract_from_json('bank_market_cap_1.json'),columns=['Name','Market Cap (US$ Billion)'])
    return extracted_data
    


<b>Question 1</b> Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [44]:
# Write your code here
df = pd.read_csv("exchange_rates.csv")


In [45]:
df.columns = ['Currency', 'Rates']
df.head(10)

,Currency,Rates
0,AUD,1.297088
1,BGN,1.608653
2,BRL,5.409196
3,CAD,1.271426
4,CHF,0.886083
5,CNY,6.483139
6,CZK,21.510117
7,DKK,6.119757
8,EUR,0.822504
9,GBP,0.732398


In [46]:
exchange_rate = df[df['Currency'] == 'GBP']['Rates'].values[0]
exchange_rate

0.7323984208000001

In [47]:
temp = pd.read_json('bank_market_cap_1.json')
temp['Market Cap (GBP$ Billion)'] = temp['Market Cap (US$ Billion)'].apply(lambda x: x* exchange_rate)
temp.head()

,Name,Market Cap (US$ Billion),Market Cap (GBP$ Billion)
0,JPMorgan Chase,390.934,286.319444
1,Industrial and Commercial Bank of China,345.214,252.834188
2,Bank of America,325.331,238.271911
3,Wells Fargo,308.013,225.588235
4,China Construction Bank,257.399,188.518621


## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1.  Changes the `Market Cap (US$ Billion)` column from USD to GBP
2.  Rounds the Market Cap (US$ Billion)\` column to 3 decimal places
3.  Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [48]:
def transform(data):
    data['Market Cap (US$ Billion)'] = data['Market Cap (US$ Billion)'].apply(lambda x: x* exchange_rate)
    data['Market Cap (US$ Billion)'] = data['Market Cap (US$ Billion)'].round(decimals = 2)
    data.rename(columns={'Market Cap (US$ Billion)':'Market Cap (GBP$ Billion)'},inplace=True)
    return data
    

In [49]:
targetfile = "bank_market_cap_gbp.csv"
logfile = "bank_market_cap_logfile.txt"

## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [50]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)
    

## Logging Function


Write the logging function <code>log</code> to log your data:


In [51]:
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y'#Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() #get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("bank_market_cap_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')
    

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [52]:
# Write your code here
log("ETL Job Started")

log("Extract phase Started")

### Extract


<code>Question 2</code> Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [53]:
extracted_data = extract()
extracted_data.head(10)


,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399
5,HSBC Holdings PLC,219.270
6,Agricultural Bank of China,203.244
7,Citigroup Inc.,203.165
8,Bank of China,181.469
9,China Merchants Bank,122.616


Log the data as <code>"Extract phase Ended"</code>


In [54]:
extracted_data = extract()
log("Extract phase Ended")

### Transform


Log the following  <code>"Transform phase Started"</code>


In [55]:
log("Transform phase Started")

<code>Question 3</code> Use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [56]:
transformed_data = transform(extracted_data)
transformed_data.head(5)


,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.32
1,Industrial and Commercial Bank of China,252.83
2,Bank of America,238.27
3,Wells Fargo,225.59
4,China Construction Bank,188.52


Log your data <code>"Transform phase Ended"</code>


In [57]:
# transformed_data = transform(extracted_data)
log("Transform phase Ended")

### Load


Log the following `"Load phase Started"`.


In [58]:
# Write your code here
log("Load phase Started")

Call the load function


In [59]:
# Write your code here
load(targetfile,transformed_data)

Log the following `"Load phase Ended"`.


In [60]:
# Write your code here
log("Load phase Ended")

log("ETL Job Ended")

## Authors


Ramesh Sannareddy, Joseph Santrcangelo and Azim Hirjani


### Other Contributors


Rav Ahuja


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
